In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypothyroidism"
cohort = "GSE224330"

# Input paths
in_trait_dir = "../../input/GEO/Hypothyroidism"
in_cohort_dir = "../../input/GEO/Hypothyroidism/GSE224330"

# Output paths
out_data_file = "../../output/preprocess/Hypothyroidism/GSE224330.csv"
out_gene_data_file = "../../output/preprocess/Hypothyroidism/gene_data/GSE224330.csv"
out_clinical_data_file = "../../output/preprocess/Hypothyroidism/clinical_data/GSE224330.csv"
json_path = "../../output/preprocess/Hypothyroidism/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyze the dataset based on the provided information
import numpy as np
import pandas as pd
import os

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: hypothyroidism (comorbidity row 3)
trait_row = 3

# Age: available in row 1
age_row = 1

# Gender: available in row 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait (hypothyroidism) data to binary"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if "hypothyroidism" in value.lower():
        return 1
    elif "none" in value.lower() or value.lower() in ["no", "healthy"]:
        return 0
    else:
        # Other comorbidities not related to hypothyroidism
        return 0

def convert_age(value):
    """Convert age data to continuous"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Remove the 'y' and convert to integer
    if 'y' in value:
        try:
            return int(value.replace('y', ''))
        except ValueError:
            return None
    else:
        try:
            return int(value)
        except ValueError:
            return None

def convert_gender(value):
    """Convert gender data to binary (0: female, 1: male)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability for filtering
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a sample characteristics dictionary based on the output from previous step
    sample_characteristics = {
        0: ['tissue: Isolated monocytes'], 
        1: ['age: 63y', 'age: 64y', 'age: 48y', 'age: 70y', 'age: 62y', 'age: 58y', 'age: 57y', 'age: 60y', 'age: 52y', 
            'age: 51y', 'age: 53y', 'age: 56y', 'age: 54y', 'age: 61y', 'age: 55y', 'age: 65y', 'age: 84y', 'age: 76y', 
            'age: 73y', 'age: 71y', 'age: 59y', 'age: 47y'], 
        2: ['gender: female', 'gender: male'], 
        3: ['comorbidity: hypothyroidism', 'comorbidity: none', 'comorbidity: osteoporosis', np.nan, 
            'comorbidity: schizoaffective disorder\xa0', 'comorbidity: arthrosis']
    }
    
    # Create a DataFrame representation of the sample characteristics
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Use the provided function to extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the selected clinical data to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import numpy as np
import gzip
import re

# Function to read GEO series matrix file for clinical data
def parse_geo_series_matrix(file_path):
    # Open the gzipped file
    with gzip.open(file_path, 'rt') as f:
        lines = f.readlines()
    
    # Extract sample characteristics data
    sample_chars = {}
    column_names = None
    current_row = None
    
    for line in lines:
        line = line.strip()
        
        # Find the line with sample names (GSM IDs)
        if line.startswith('!Sample_geo_accession'):
            column_names = line.split('\t')[1:]  # Skip the first element (header)
        
        # Start capturing sample characteristics
        elif line.startswith('!Sample_characteristics_ch'):
            current_row = len(sample_chars)
            parts = line.split('\t')
            values = parts[1:]
            sample_chars[current_row] = values
        
        # Stop when we reach the data table
        elif line.startswith('!series_matrix_table_begin'):
            break
    
    # Convert to DataFrame
    if column_names and sample_chars:
        df = pd.DataFrame(sample_chars, index=column_names).T
        return df
    
    return None

# Scan the cohort directory to understand available files
series_matrix_file = os.path.join(in_cohort_dir, "GSE224330_series_matrix.txt.gz")
is_gene_available = os.path.exists(series_matrix_file)  # Series matrix usually contains gene expression data

# Parse the series matrix file to extract clinical data
clinical_data = None
if is_gene_available:
    clinical_data = parse_geo_series_matrix(series_matrix_file)
    if clinical_data is not None:
        print("Successfully parsed clinical data from series matrix file")
    else:
        print("Could not parse clinical data from series matrix file")
else:
    print("No series matrix file found for gene expression data")

# Inspect clinical data to find trait, age, and gender information
sample_chars = {}
if clinical_data is not None:
    # Print unique values for each row to help identify trait, age, and gender
    print("\nSample characteristics:")
    for i, row in clinical_data.iterrows():
        unique_values = row.unique()
        sample_chars[i] = unique_values
        if len(unique_values) < 10:  # Only print if there aren't too many unique values
            print(f"Row {i}: {unique_values}")
else:
    print("No clinical data available to analyze")

# Now let's identify trait, age, and gender rows and define conversion functions
trait_row = None
age_row = None
gender_row = None

# Default conversion functions
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if any(term in value_str for term in ["hypothyroidism", "hypothyroid", "hashimoto"]):
        return 1  # Has hypothyroidism
    elif any(term in value_str for term in ["healthy", "control", "normal", "euthyroid"]):
        return 0  # No hypothyroidism
    else:
        return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Try to extract numeric age
    age_match = re.search(r'(\d+(?:\.\d+)?)', value_str)
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if any(term in value_str for term in ["female", "f", "woman", "women"]):
        return 0  # Female
    elif any(term in value_str for term in ["male", "m", "man", "men"]):
        return 1  # Male
    else:
        return None

# Check for trait, age, and gender in sample characteristics
if sample_chars:
    for row, values in sample_chars.items():
        values_str = ' '.join([str(v) for v in values if v is not None])
        
        # Look for trait information
        if trait_row is None and any(term in values_str.lower() for term in 
                                   ["thyroid", "disease", "diagnosis", "status", "condition", "patient", "subject"]):
            trait_row = row
        
        # Look for age information
        if age_row is None and any(term in values_str.lower() for term in ["age", "years", "year old"]):
            age_row = row
        
        # Look for gender information
        if gender_row is None and any(term in values_str.lower() for term in ["gender", "sex", "male", "female"]):
            gender_row = row

print(f"\nIdentified rows - Trait: {trait_row}, Age: {age_row}, Gender: {gender_row}")

# Check if we need to refine our search or use more complex matching
if trait_row is None and sample_chars:
    print("Trait row not identified. Attempting deeper analysis...")
    for row, values in sample_chars.items():
        # Check each value in the row to see if it might indicate disease status
        has_disease_indicators = False
        has_control_indicators = False
        
        for val in values:
            if val is not None:
                val_str = str(val).lower()
                if any(term in val_str for term in ["thyroid", "hypothyroid", "hashimoto", "disease", "patient"]):
                    has_disease_indicators = True
                if any(term in val_str for term in ["healthy", "control", "normal"]):
                    has_control_indicators = True
        
        if has_disease_indicators or has_control_indicators:
            trait_row = row
            print(f"Potential trait row identified at row {row}")
            break

# Initial validation of dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# Extract clinical features if available
if is_trait_available and clinical_data is not None:
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Extract clinical features
    selected_clinical_data = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the result
    preview = preview_df(selected_clinical_data)
    print("\nClinical data preview:")
    print(preview)
    
    # Save the clinical data
    selected_clinical_data.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print(f"Clinical feature extraction skipped: trait_available={is_trait_available}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 5: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# The gene IDs shown (e.g., 'A_19_P00315452') appear to be Agilent microarray probe IDs,
# not standard human gene symbols (which would look like "BRCA1", "TP53", etc.)
# These probe IDs need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in the gene annotation dataframe correspond to the gene expression identifiers
# Looking at the gene expression data, we see identifiers like 'A_19_P00315452'
# In the gene annotation dataframe, the column 'ID' appears to store the probe IDs

# Looking at the column names in the gene annotation data:
# ['ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE', 'SPOT_ID']

# The 'ID' column stores the probe identifiers, and 'GENE_SYMBOL' stores the gene symbols
# Note: In the gene annotation, the first few rows are control probes without gene symbols

# 2. Get a gene mapping dataframe
print("Creating gene mapping from probe IDs to gene symbols...")
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping with {len(mapping_df)} rows")
print("Sample of mapping data:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
print("Converting probe-level measurements to gene-level expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted to gene expression data with {len(gene_data.index)} unique genes")
print("Sample of gene expression data (first few genes and samples):")
print(preview_df(gene_data))

# Verify the data has valid gene symbols
print(f"First 10 gene symbols in the processed data:")
print(list(gene_data.index[:10]))

# Normalize gene symbols in index
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data.index)} unique genes")
print("First 10 normalized gene symbols:")
print(list(gene_data.index[:10]))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
import os

# 1. Load the normalized gene data and clinical data
# Note: Gene data was already normalized and saved in Step 7
print("Loading gene expression and clinical data...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)

print(f"Loaded gene data with {len(gene_data.index)} genes and {len(gene_data.columns)} samples")
print(f"Loaded clinical data with {len(clinical_data.index)} features and {len(clinical_data.columns)} samples")

# Debug: Inspect sample IDs in both datasets
print("\nSample ID format inspection:")
print("Gene data column names sample:", list(gene_data.columns)[:3])
print("Clinical data index sample:", list(clinical_data.index)[:3])

# Standardize sample IDs to ensure consistent formatting
print("\nStandardizing sample IDs...")
gene_data.columns = gene_data.columns.str.strip('"')
clinical_data.index = clinical_data.index.str.strip('"')

# Verify after standardization
print("Gene data column names after standardization:", list(gene_data.columns)[:3])
print("Clinical data index after standardization:", list(clinical_data.index)[:3])

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data)
print(f"Linked data: {linked_data.shape[0]} samples, {linked_data.shape[1]} features")
print("Preview of linked data:")
print(preview_df(linked_data))

# 3. Handle missing values in the linked data
print("\nHandling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values: {linked_data.shape[0]} samples, {linked_data.shape[1]} features")

# Check if we have any samples left after handling missing values
if linked_data.shape[0] == 0:
    print("No samples remaining after handling missing values. Dataset cannot be used.")
    is_biased = True  # Set to True to mark dataset as unusable
    is_usable = False
    
    # Add note about the issue
    note = "No samples remaining after handling missing values. The dataset has too many missing values in gene expression data."
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )
    
    print(f"\nDataset not usable for {trait} analysis due to excessive missing values. Data not saved.")
else:
    # 4. Determine whether the trait and demographic features are biased
    print("\nChecking for bias in trait and demographic features...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 5. Final quality validation and save metadata
    print("\nPerforming final quality validation...")
    note = "Successfully linked gene expression and clinical data for Hypothyroidism analysis."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        
        # Save the linked data
        linked_data.to_csv(out_data_file)
        print(f"\nLinked data saved to {out_data_file}")
    else:
        print(f"\nDataset not usable for {trait} analysis due to data quality issues. Data not saved.")